In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
import re
import collections
import random
import math

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
pd.set_option('display.max_column', None)
train_input = pd.read_csv('train.csv')
test_input = pd.read_csv('test.csv')
print("train_input shape is: ", np.shape(train_input))
print("test_input shape is: ", np.shape(test_input))

train_input shape is:  (4743, 2)
test_input shape is:  (1701, 2)


In [3]:
def label_tweet(input_set):
    handle = input_set['handle']
    # put it into an array named label, where 0 represents HillaryClinton, 1 represents readDonaldTrump
    label = []
    for i in range(len(handle)):
        if handle[i] == "HillaryClinton":
            label.append(0)
        if handle[i] == "realDonaldTrump":
            label.append(1)
    label = np.asarray(label)
    return label

train_label = label_tweet(train_input)
print(train_label[:10])

[0 0 0 0 1 0 0 0 1 1]


In [4]:
train_corpus = train_input['tweet'].as_matrix()
print(np.shape(train_corpus))
print(train_corpus[:5])
test_corpus = test_input['tweet'].as_matrix()
print(np.shape(test_corpus))

(4743,)
[ 'The question in this election: Who can put the plans into action that will make your life better? https://t.co/XreEY9OicG'
 'Last night, Donald Trump said not paying taxes was "smart." You know what I call it? Unpatriotic. https://t.co/t0xmBfj7zF'
 "If we stand together, there's nothing we can't do. \n\nMake sure you're ready to vote: https://t.co/tTgeqxNqYm https://t.co/Q3Ymbb7UNy"
 "Both candidates were asked about how they'd confront racial injustice. Only one had a real answer. https://t.co/sjnEokckis"
 'Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:… https://t.co/dfzsbICiXc']
(1701,)


In [5]:
# Load the stopwords
stop_words = stopwords.words('english')
# 'https' seems useless, so I add it to stop_words
stop_words.append(u'https')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [6]:
# Tokenize
def tokenization(text):
#     return [i for i in text]
    tokens=[]
#     for word in nltk.word_tokenize(text.decode('utf-8')):
    for word in nltk.word_tokenize(text):
        # skip all the websites, punctuations, pure digits
        if not re.match('[//]', word) and re.search('[a-zA-Z]', word) and word.lower() not in stop_words:
            tokens.append(word.lower())
    return tokens

# Tokenize training set
train_corpus_tokenized = []
for i in train_corpus:
    train_corpus_tokenized.append(' '.join(tokenization(i)))
    
print(len(train_corpus_tokenized))
print(train_corpus_tokenized[:5])

# Tokenize testing set
test_corpus_tokenized = []
for i in test_corpus:
    test_corpus_tokenized.append(' '.join(tokenization(i)))

print(len(test_corpus_tokenized))
print(test_corpus_tokenized[:5])

4743
['question election put plans action make life better', 'last night donald trump said paying taxes smart know call unpatriotic', "stand together 's nothing ca n't make sure 're ready vote", "candidates asked 'd confront racial injustice one real answer", 'join 3pm rally tomorrow mid-america center council bluffs iowa tickets']
1701
["could n't proud hillaryclinton vision command last night 's debate showed 's ready next potus", "election important sit go make sure 're registered nationalvoterregistrationday -h", 'government people join movement today', "national voterregistrationday make sure 're registered vote makeamericagreatagain…", 'great afternoon little havana hispanic community leaders thank support imwithyou']


In [7]:
train_tokenized_word = []
for i in range(len(train_corpus_tokenized)):
    train_tokenized_word.append(tf.compat.as_str(train_corpus_tokenized[i]).split())
print(len(train_tokenized_word))
print(train_tokenized_word[:5])

vocabulary_size = 10000

test_tokenized_word = []
for i in range(len(test_corpus_tokenized)):
    test_tokenized_word.append(tf.compat.as_str(test_corpus_tokenized[i]).split())
print(len(test_tokenized_word))
print(test_tokenized_word[:5])

4743
[['question', 'election', 'put', 'plans', 'action', 'make', 'life', 'better'], ['last', 'night', 'donald', 'trump', 'said', 'paying', 'taxes', 'smart', 'know', 'call', 'unpatriotic'], ['stand', 'together', "'s", 'nothing', 'ca', "n't", 'make', 'sure', "'re", 'ready', 'vote'], ['candidates', 'asked', "'d", 'confront', 'racial', 'injustice', 'one', 'real', 'answer'], ['join', '3pm', 'rally', 'tomorrow', 'mid-america', 'center', 'council', 'bluffs', 'iowa', 'tickets']]
1701
[['could', "n't", 'proud', 'hillaryclinton', 'vision', 'command', 'last', 'night', "'s", 'debate', 'showed', "'s", 'ready', 'next', 'potus'], ['election', 'important', 'sit', 'go', 'make', 'sure', "'re", 'registered', 'nationalvoterregistrationday', '-h'], ['government', 'people', 'join', 'movement', 'today'], ['national', 'voterregistrationday', 'make', 'sure', "'re", 'registered', 'vote', 'makeamericagreatagain…'], ['great', 'afternoon', 'little', 'havana', 'hispanic', 'community', 'leaders', 'thank', 'support',

In [8]:
cnt = collections.Counter()
for i in range(len(train_tokenized_word)):
    for word in train_tokenized_word[i]:
        cnt[word] += 1

print(len(cnt))

8507


In [9]:
def build_dataset(cnt, words, n_words):
    """Process raw inputs into a dataset."""
    count = [['UNK', -1]]
    count.extend(cnt.most_common(n_words - 1))
#     print count[:20]
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = []
    data_all = []
    unk_count = 0
    for i in range(len(words)):
        inner_data = []
        for word in words[i]:
            index = dictionary.get(word, 0)
            if index == 0:  # dictionary['UNK']
                unk_count += 1
            inner_data.append(index)
            data_all.append(index)
        data.append(inner_data)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, data_all, count, dictionary, reversed_dictionary

train_x, data_all, count, dictionary, reverse_dictionary = build_dataset(cnt, train_tokenized_word, vocabulary_size)

In [10]:
test = []
for sentence in test_tokenized_word:
    cur = []
    for word in sentence:
        if(word in dictionary):
            cur.append(dictionary[word])
        else:
            cur.append(0)
    test.append([cur,[0, 0]])

print(len(test))
print(test[:5])
test_length = len(test)
print(test_length)

1701
[[[92, 6, 120, 56, 1150, 0, 44, 83, 2, 99, 1387, 2, 199, 162, 32], [0, 0]], [[104, 271, 1683, 51, 12, 145, 75, 600, 1661, 82], [0, 0]], [[1341, 9, 43, 207, 26], [0, 0]], [[116, 0, 12, 145, 75, 600, 22, 0], [0, 0]], [[5, 914, 241, 0, 2898, 622, 497, 4, 68, 169], [0, 0]]]
1701


In [11]:
train_all = [[train_x[i], [train_label[i], 1-train_label[i]]] for i in range(0, len(train_x))]
# train_all = [[train_x[i], [train_label[i]]] for i in range(0, len(train_x))]

r_index = list(range(len(train_all)))
random.shuffle(r_index)
train = [train_all[i] for i in r_index[:int(len(r_index)*0.8)]]
valid = [train_all[i] for i in r_index[int(len(r_index)*0.8):]]

print(len(train))
print(train[:5])
print(len(valid))
print(valid[:5])
print(len(count))

3794
[[[25, 9, 6517, 1268, 1, 364, 71, 1018, 326], [0, 1]], [[32, 325, 1945, 736, 64, 860, 45, 23, 57, 16], [0, 1]], [[1920, 2, 195, 25, 33, 41, 856, 3264, 23, 9, 16], [0, 1]], [[814, 911, 8, 679, 570, 1749, 2475, 6425, 6426, 6427], [1, 0]], [[424, 33, 2097, 1328, 84, 3371, 1513, 357, 507, 6171, 175, 158, 58], [0, 1]]]
949
[[[314, 339, 578, 2425, 1406, 1688, 44, 66, 8, 2, 1403], [0, 1]], [[2, 447, 603, 261, 323, 56, 32, 914, 999, 5744], [0, 1]], [[376, 312], [0, 1]], [[127, 7419, 3604, 35, 1957, 2223, 140, 3603, 7420, 964, 103, 7421], [1, 0]], [[7572, 8, 190, 2, 957, 7573, 93, 62, 6, 158], [1, 0]]]
8508


In [12]:
class SimpleDataIterator():
    def __init__(self, df):
        self.df = df
        self.size = len(self.df)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        random.shuffle(self.df)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            print("SimpleDataIterator epoch : ", self.epochs)
            self.shuffle()
        res = self.df[self.cursor:self.cursor+n]
        self.cursor += n
        return res

In [13]:
data = SimpleDataIterator(valid)
d = data.next_batch(500)
print('Input sequences : ', d[:5])

Input sequences :  [[[6176, 8, 601, 12, 23, 171, 3, 11, 55, 1408], [1, 0]], [[640, 5207, 637, 5208, 163, 244, 58, 351], [0, 1]], [[1, 294, 1093, 764, 541, 451, 1488, 823, 149, 4070, 442, 4071], [0, 1]], [[103, 180, 4519, 127, 644], [0, 1]], [[1264, 85, 165, 180, 58, 36, 54, 316, 7498, 77, 33, 5, 1075, 859, 1917], [1, 0]]]


In [14]:
class PaddedDataIterator(SimpleDataIterator):
    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df[self.cursor:self.cursor+n]
        self.cursor += n

        # Pad sequences with 0s so they are all the same length
        max_len = 0
        for row in res:
            if len(row[0]) > max_len:
                max_len = len(row[0])
        seqlen = np.array([max_len for i in range(len(res))])
        ret = []
        label = []
        for row in res:
            ret += [row[0] + [0]*(max_len-len(row[0]))]
            label.append(row[1][0])
        x = np.array(ret)
        y = np.array(label)

        return x, y, seqlen

In [15]:
data = PaddedDataIterator(train)
d = data.next_batch(3)
print('Input sequences\n', d[0])
print(d[0].shape)
print(d[1])
print(d[1].shape)
print(d[2])

Input sequences
 [[ 587  535  812 1162  779 4325  404  398  385  588  932  818    0    0
     0]
 [1944  481  594 8071   18  189 8072  551 3189  238 8073  371 2567  485
   282]
 [ 480   15  735    0    0    0    0    0    0    0    0    0    0    0
     0]]
(3, 15)
[1 1 1]
(3,)
[15 15 15]


In [16]:
batch_size = 256

In [17]:
def align(data):
    print(len(data))
    max_len = 0
    for row in data:
        if len(row[0]) > max_len:
            max_len = len(row[0])
    print(max_len)
    ret = []
    label = []
    for row in data:
        ret += [row[0] + [0]*(max_len-len(row[0]))]
        label.append(row[1][0])
    x = np.array(ret)
    y = np.array(label)
    seq_len = np.array([max_len for i in data])
    
    return x, y, seq_len

# def xx():
#     max_len = 0
#     for row in res:
#         if len(row[0]) > max_len:
#             max_len = len(row[0])
#     seqlen = np.array([max_len for i in range(len(res))])
#     ret = []
#     label = []
#     for row in res:
#         ret += [row[0] + [0]*(max_len-len(row[0]))]
#         label.append(row[1][0])
#     x = np.array(ret)
#     y = np.array(label)

# test_length = 1701
print(test_length)
print(batch_size)
print(test_length/batch_size)

test_list = []
test_addlen = test
test_addlen.extend(test[0:batch_size])
# for i in range(test_length/batch_size+1):
for i in range(math.ceil(test_length/batch_size)):
    print(i)
    x, y, seq_len = align(test[i*batch_size:(i+1)*batch_size])
    print(x.shape, ", ", y.shape, ", ", seq_len.shape)
#     print(y.shape)
#     print(seq_len.shape)
    test_list.append([x, y, seq_len])

print("=====")
print(len(test_list[0]))
    

# test_align, max_len = align(test)
# print(len(test_align))

1701
256
6.64453125
0
256
19
(256, 19) ,  (256,) ,  (256,)
1
256
18
(256, 18) ,  (256,) ,  (256,)
2
256
23
(256, 23) ,  (256,) ,  (256,)
3
256
20
(256, 20) ,  (256,) ,  (256,)
4
256
20
(256, 20) ,  (256,) ,  (256,)
5
256
18
(256, 18) ,  (256,) ,  (256,)
6
256
19
(256, 19) ,  (256,) ,  (256,)
=====
3


In [36]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

NUM_LAYERS=2

def gruCell(state_size):
#     lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, state_is_tuple=True)
    gru_cell = tf.nn.rnn_cell.GRUCell(state_size)
    return gru_cell
    
def build_graph(
    vocab_size = len(dictionary),
    state_size = 128,
    batch_size = 256,
    num_classes = 2):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
#     keep_prob = tf.constant(1.0)
#     print("====",x.shape)
#     print("====",y.shape)
#     print("====",seqlen.shape)

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

#     # RNN
#     cell = tf.nn.rnn_cell.GRUCell(state_size)
# #     cell = tf.nn.rnn_cell.BasicLSTMCell(state_size)
# #     cell = tf.contrib.rnn.MultiRNNCell([gruCell(state_size) for _ in range(NUM_LAYERS)])
#     init_state = tf.get_variable('init_state', [1, state_size],
#                                  initializer=tf.constant_initializer(0.0))
#     init_state = tf.tile(init_state, [batch_size, 1])
#     rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, x, sequence_length=seqlen,
#                                                  initial_state=init_state)

    # RNN
    cell = tf.nn.rnn_cell.GRUCell(state_size)
    init_state = tf.get_variable('init_state', [1, state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                 initial_state=init_state)

    # Add dropout, as the model otherwise quickly overfits
#     rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)

    """
    Obtain the last relevant output. The best approach in the future will be to use:

        last_rnn_output = tf.gather_nd(rnn_outputs, tf.pack([tf.range(batch_size), seqlen-1], axis=1))

    which is the Tensorflow equivalent of numpy's rnn_outputs[range(30), seqlen-1, :], but the
    gradient for this op has not been implemented as of this writing.

    The below solution works, but throws a UserWarning re: the gradient.
    """
    idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)

    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
#     print(correct.shape)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
#     print(accuracy.shape)

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
#         'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }

In [37]:
def train_graph(graph, batch_size = 256, num_epochs = 50, iterator = PaddedDataIterator):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        tr = iterator(train)
        tv = iterator(valid)
#         te = iterator(test)

        step, accuracy = 0, 0
        tr_losses, tv_losses = [], []
        current_epoch = 0
        while current_epoch < num_epochs:
            step += 1
            batch = tr.next_batch(batch_size)
#             print("----",batch[0].shape)
#             print("----",batch[1].shape)
#             print("----",batch[2].shape)
#             dropout_parameter=np.array(0.6)
#             print("----",dropout_parameter.shape)
#             feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: dropout_parameter}
            feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
            accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
            accuracy += accuracy_

            if tr.epochs > current_epoch:
                current_epoch += 1
                tr_losses.append(accuracy / step)
                step, accuracy = 0, 0

                #eval test set
                tv_epoch = tv.epochs
                while tv.epochs == tv_epoch:
                    step += 1
                    batch = tv.next_batch(batch_size)
                    feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                    accuracy_ = sess.run([g['accuracy']], feed_dict=feed)[0]
                    accuracy += accuracy_

                tv_losses.append(accuracy / step)
                step, accuracy = 0,0
                print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1], "- tv:", tv_losses[-1])
            
        predictions = []
        for te in test_list:
            feed = {g['x']: te[0], g['y']: te[1], g['seqlen']: te[2]}
            preds_, _ = sess.run([g['preds'], g['ts']], feed_dict=feed)
            print(len(preds_))
            predictions.extend(preds_)

    return tr_losses, tv_losses, predictions

In [38]:
g = build_graph()
tr_losses, tv_losses, predictions = train_graph(g)
print(len(predictions))

/home/ziyi/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Accuracy after epoch 1  - tr: 0.508333333333 - tv: 0.4873046875
Accuracy after epoch 2  - tr: 0.502511160714 - tv: 0.514322916667
Accuracy after epoch 3  - tr: 0.537388392857 - tv: 0.654947916667
Accuracy after epoch 4  - tr: 0.871930803571 - tv: 0.83203125
Accuracy after epoch 5  - tr: 0.924107142857 - tv: 0.8671875
Accuracy after epoch 6  - tr: 0.932756696429 - tv: 0.885416666667
Accuracy after epoch 7  - tr: 0.942801339286 - tv: 0.864583333333
Accuracy after epoch 8  - tr: 0.946149553571 - tv: 0.888020833333
Accuracy after epoch 9  - tr: 0.954520089286 - tv: 0.89453125
Accuracy after epoch 10  - tr: 0.967912946429 - tv: 0.890625
Accuracy after epoch 11  - tr: 0.976004464286 - tv: 0.90234375
Accuracy after epoch 12  - tr: 0.984933035714 - tv: 0.899739583333
Accuracy after epoch 13  - tr: 0.990513392857 - tv: 0.91015625
Accuracy after epoch 14  - tr: 0.992466517857 - tv: 0.889322916667
Accuracy after epoch 15  - tr: 0.994698660714 - tv: 0.90234375
Accuracy after epoch 16  - tr: 0.9955

In [37]:
# print(predictions[:test_length])
print(predictions[:50])

[array([  9.99783456e-01,   2.16587825e-04], dtype=float32), array([  9.99985099e-01,   1.49040734e-05], dtype=float32), array([  4.22566445e-05,   9.99957800e-01], dtype=float32), array([  9.99928355e-01,   7.15992355e-05], dtype=float32), array([  2.82522287e-05,   9.99971747e-01], dtype=float32), array([  9.99980330e-01,   1.97034951e-05], dtype=float32), array([ 0.0166364 ,  0.98336363], dtype=float32), array([  9.99961138e-01,   3.88448861e-05], dtype=float32), array([  9.99980211e-01,   1.98258767e-05], dtype=float32), array([ 0.84958726,  0.15041277], dtype=float32), array([  9.99469578e-01,   5.30452642e-04], dtype=float32), array([  9.99907732e-01,   9.22226100e-05], dtype=float32), array([  9.99968529e-01,   3.14640856e-05], dtype=float32), array([ 0.91769451,  0.08230553], dtype=float32), array([ 0.61804193,  0.3819581 ], dtype=float32), array([ 0.01473562,  0.98526436], dtype=float32), array([  9.99910355e-01,   8.96546262e-05], dtype=float32), array([  1.98048274e-05,   9.

In [173]:
import csv
with open('csvtest4.csv', 'w') as csvfile:
    csvfile.write(",".join(['id', 'realDonaldTrump', 'HillaryClinton'])+"\n")
    data = []
    for i in range(test_length):
        csvfile.write(",".join([str(i), str(predictions[i][1]), str(predictions[i][0])])+"\n")
